In [3]:
import warnings
warnings.filterwarnings('ignore')

import geopandas as gpd
from shapely import geometry
import pandas as pd

In [6]:
gdf = gpd.read_file("VA_Overlap.json")
gdf.head(2)

,CName1,CName2,ID,geometry
0,va-albemarle-0021,va-charlottesvillecity-0354,va-overlap-0,"MULTIPOLYGON (((-78.46688 38.06782, -78.46687 ..."
1,va-albemarle-0022,va-augusta-0169,va-overlap-1,"MULTIPOLYGON (((-78.83475 38.05003, -78.83458 ..."


In [7]:
Polygon = geometry.polygon.Polygon
MultiPolygon = geometry.multipolygon.MultiPolygon

In [11]:
def parse(gdf, arg_i, arg_shape_id, arg_polygon_id):
    columns = ["geovertex_id", "shape_id", "polygon_id", "ID", "CName1", "CName2", "x_pos", "y_pos"]
    df = pd.DataFrame(columns=columns)

    counter = 0
    i = arg_i
    shape_id = arg_shape_id
    polygon_id = arg_polygon_id

    for index, row in gdf.iterrows():
        # for visualization only
        if counter%15==0:
            print(" {:.2f}% ".format(counter/len(gdf)*100), end="")
        else:
            print("#", end="")

        cur_shape = row["geometry"]
        cur_type = type(cur_shape)

        row = row.drop(["geometry"])
        cur_row = row
        cur_row["shape_id"] = shape_id
        shape_id += 1

        if cur_type == MultiPolygon:
            for polygon in cur_shape:
                cur_row["polygon_id"] = polygon_id
                polygon_id += 1
                for coords in polygon.exterior.coords:
                    cur_row["geovertex_id"] = i
                    i += 1
                    cur_row["x_pos"] = coords[0]
                    cur_row["y_pos"] = coords[1]
                    df = df.append(cur_row, ignore_index=True)
                    cur_row = row
        elif cur_type == Polygon:
            cur_row["polygon_id"] = polygon_id
            polygon_id += 1
            for coords in cur_shape.exterior.coords:
                cur_row["geovertex_id"] = i
                i += 1
                cur_row["x_pos"] = coords[0]
                cur_row["y_pos"] = coords[1]
                df = df.append(cur_row, ignore_index=True)
                cur_row = row
        counter += 1
        
    return df

In [12]:
next_i, next_shape, next_poly = 1495449, 959, 1940

# parsing
gdf_Parsed = parse(gdf, arg_i=next_i, arg_shape_id=next_shape, arg_polygon_id=next_poly)

# saving the result 
gdf_Parsed.to_csv("VA_Overlap_Parsed.csv", index=False)

 0.00% ############## 1.54% #